In [1]:
import numpy as np
import os, requests
import pandas as pd
import tarfile
from src.file_ops import npy_loader
from src.session_metadata import Session
from dataclasses import dataclass
from typing import Optional
from pathlib import Path
import matplotlib.pyplot as plt


In [2]:
# Set the current working directory
cwd = os.chdir(r"C:\Python Work Directory\NMA_Impact_Scholars_Steinmetz")

# @title Data retrieval
data_directory = r'data\spikeAndBehavioralData'

session_label_string = "Cori_2016-12-14"

session_label = session_label_string.split("_")
print(session_label)

# test_dataset
test_LFP = session_label_string

['Cori', '2016-12-14']


In [5]:
#Uncomment this section to enable the functions below


# from abc import ABC, abstractmethod
# from dataclasses import dataclass, field
# from pathlib import Path
# import tarfile
# import numpy as np
# from typing import Dict, Optional
# from src.file_ops import npy_loader
# import pandas as pd
#
# def calculate_peak_to_trough_duration(waveforms, sampling_rate=30000):
#     """
#     Calculate the peak-to-trough duration of a waveform.
#
#     Parameters:
#     waveform (np.ndarray): Template waveform array
#     sampling_rate (int): Sampling rate in Hz, default 30000 for typical ephys
#
#     Returns:
#     float: Peak-to-trough duration in milliseconds
#     """
#     # Find peak and trough indices
#     waveform = waveforms[:,0]
#
#     peak_idx = np.argmax(waveform)
#     trough_idx = np.argmin(waveform)
#
#     # Calculate time difference
#     time_diff_samples = abs(peak_idx - trough_idx)
#
#     # Convert to milliseconds
#     duration_ms = (time_diff_samples / sampling_rate) * 1000
#
#     return duration_ms
#
# @dataclass
# class BaseLoader(ABC):
#     """Abstract base class for loading data from tar archives."""
#
#     @classmethod
#     @abstractmethod
#     def get_file_mapping(cls) -> Dict[str, str]:
#         """Subclasses must define the mapping of tar filenames to attributes."""
#         pass
#
#     @classmethod
#     def from_tar(cls, tar_path: str | Path) -> 'BaseLoader':
#         """Load data from a tar archive."""
#         instance = cls()
#         file_attr_map = cls.get_file_mapping()
#
#         with tarfile.open(tar_path, 'r') as tar:
#             for file_name, attr_name in file_attr_map.items():
#                 if file_name not in tar.getnames():
#                     print(f"Warning: {file_name} not found in tar archive")
#                     continue
#                 setattr(instance, attr_name, np.squeeze(npy_loader(tar, file_name)))
#
#         return instance
#
#     def to_dataframe(self) -> pd.DataFrame:
#         """
#         Convert the Trials data to a pandas DataFrame.
#         For multi-dimensional arrays, only the first dimension is used as the index,
#         and the remaining dimensions are stored as array objects in the cells.
#         """
#         data_dict = {}
#         base_length = None
#
#         # Process each attribute
#         for attr_name, value in self.__dict__.items():
#             print(attr_name, value.shape)
#             if value is not None:
#                 if len(value.shape) == 1:
#                     # 1D arrays can be directly added
#                     data_dict[attr_name] = value
#                     if base_length is None:
#                         base_length = len(value)
#                 else:
#                     # For multi-dimensional arrays, store them as objects
#                     # Each row will contain a slice of the array
#                     data_dict[attr_name] = [value[i] for i in range(value.shape[0])]
#                     if base_length is None:
#                         base_length = value.shape[0]
#
#         # Create DataFrame
#         df = pd.DataFrame(data_dict)
#
#         return df
#
#
#
# @dataclass
# class Clusters(BaseLoader):
#     """Represents cluster-related data."""
#     depths: Optional[np.ndarray] = field(default=None)
#     original_ids: Optional[np.ndarray] = field(default=None)
#     site: Optional[np.ndarray] = field(default=None)
#     probe: Optional[np.ndarray] = field(default=None)
#     template_waveform_chans: Optional[np.ndarray] = field(default=None)
#     template_waveforms: Optional[np.ndarray] = field(default=None)
#     waveform_duration: Optional[np.ndarray] = field(default=None)
#     phy_annotation: Optional[np.ndarray] = field(default=None)
#
#     def __post_init__(self):
#         """Ensure numeric fields are of integer type where applicable."""
#         for attr in ["original_ids", "site", "probe"]:
#             value = getattr(self, attr, None)
#             if value is not None:
#                 setattr(self, attr, value.astype(int))
#
#     @classmethod
#     def get_file_mapping(cls) -> Dict[str, str]:
#         return {
#             'clusters.depths.npy': 'depths',
#             'clusters.originalIDs.npy': 'original_ids',
#             'clusters.peakChannel.npy': 'site',
#             'clusters.probes.npy': 'probe',
#             'clusters.templateWaveformChans.npy': 'template_waveform_chans',
#             'clusters.templateWaveforms.npy': 'template_waveforms',
#             'clusters.waveformDuration.npy': 'waveform_duration',
#             'clusters._phy_annotation.npy': 'phy_annotation',
#         }
#
#     def to_dataframe(self) -> pd.DataFrame:
#         df = super().to_dataframe()
#         df['peak_to_trough_duration'] = df['template_waveforms'].apply(calculate_peak_to_trough_duration)
#
#         return df
#
# @dataclass
# class Trials(BaseLoader):
#     """Represents trial-related data."""
#     feedback_type: Optional[np.ndarray] = field(default=None)
#     feedback_times: Optional[np.ndarray] = field(default=None)
#     gocue_times: Optional[np.ndarray] = field(default=None)
#     included: Optional[np.ndarray] = field(default=None)
#     intervals: Optional[np.ndarray] = field(default=None)
#     repNum: Optional[np.ndarray] = field(default=None)
#     response_choice: Optional[np.ndarray] = field(default=None)
#     response_times: Optional[np.ndarray] = field(default=None)
#     contrast_left: Optional[np.ndarray] = field(default=None)
#     contrast_right: Optional[np.ndarray] = field(default=None)
#     stimulus_times: Optional[np.ndarray] = field(default=None)
#
#     @classmethod
#     def get_file_mapping(cls) -> Dict[str, str]:
#         return {
#             'trials.feedbackType.npy': 'feedback_type',
#             'trials.feedback_times.npy': 'feedback_times',
#             'trials.goCue_times.npy': 'gocue_times',
#             'trials.included.npy': 'included',
#             'trials.intervals.npy': 'intervals',
#             'trials.repNum.npy': 'repNum',
#             'trials.response_choice.npy': 'response_choice',
#             'trials.response_times.npy': 'response_times',
#             'trials.visualStim_contrastLeft.npy': 'contrast_left',
#             'trials.visualStim_contrastRight.npy': 'contrast_right',
#             'trials.visualStim_times.npy': 'stimulus_times',
#         }
#
#     def to_dataframe(self) -> pd.DataFrame:
#         df = super().to_dataframe()
#         df['start'] = df.intervals.apply(lambda x: x[0])
#         df['end'] = df.intervals.apply(lambda x: x[1])
#
#         return df
#
# @dataclass
# class Spikes(BaseLoader):
#     """Represents spike-related data."""
#     amps: Optional[np.ndarray] = field(default=None)
#     times: Optional[np.ndarray] = field(default=None)
#     clusters: Optional[np.ndarray] = field(default=None)
#     depths: Optional[np.ndarray] = field(default=None)
#
#     @classmethod
#     def get_file_mapping(cls) -> Dict[str, str]:
#         return{
#             'spikes.amps.npy': 'amps',
#             'spikes.times.npy': 'times',
#             'spikes.clusters.npy': 'clusters',
#             'spikes.depths.npy': 'depths',
#         }
#
#     def to_dataframe(self) -> pd.DataFrame:
#         """
#         Convert the Spikes data to a pandas DataFrame.
#         For multi-dimensional arrays, only the first dimension is used as the index,
#         and the remaining dimensions are stored as array objects in the cells.
#         """
#
#         # Create DataFrame
#         df = super().to_dataframe()
#         df['clusters'] = df['clusters'].astype(int)
#         return df
#
#
# @dataclass
# class Channels(BaseLoader):
#     """Represents channel-related data."""
#     brain_location: Optional[pd.DataFrame] = None
#     site_positions: Optional[np.ndarray] = field(default=None)
#     site: Optional[np.ndarray] = field(default=None)
#     probe: Optional[np.ndarray] = field(default=None)
#     raw_row: Optional[np.ndarray] = field(default=None)
#
#     @classmethod
#     def get_file_mapping(cls) -> Dict[str, str]:
#         return {
#             'channels.sitePositions.npy': 'site_positions',
#             'channels.probe.npy': 'probe',
#             'channels.site.npy': 'site',
#             'channels.rawRow.npy': 'raw_row',
#             'channels.brainLocation.tsv': 'brain_location',
#         }
#
#     @classmethod
#     def from_tar(cls, tar_path: str | Path) -> 'BaseLoader':
#         """Load data from a tar archive."""
#         instance = cls()
#         file_attr_map = cls.get_file_mapping()
#
#         with tarfile.open(tar_path, 'r') as tar:
#             for file_name, attr_name in file_attr_map.items():
#                 if file_name not in tar.getnames():
#                     print(f"Warning: {file_name} not found in tar archive")
#                     continue
#                 if attr_name == 'brain_location':
#                     setattr(instance, attr_name, pd.read_csv(tar.extractfile(file_name), sep='\t'))
#                 else:
#                     setattr(instance, attr_name, np.squeeze(npy_loader(tar, file_name)))
#         return instance
#
#     def to_dataframe(self) -> pd.DataFrame:
#         """
#         Convert the Trials data to a pandas DataFrame.
#         For multi-dimensional arrays, only the first dimension is used as the index,
#         and the remaining dimensions are stored as array objects in the cells.
#         """
#         data_dict = {}
#         base_length = None
#
#         # Process each attribute
#         for attr_name, value in self.__dict__.items():
#             print(attr_name, value.shape)
#             if attr_name == 'brain_location':
#                 continue
#             elif value is not None:
#                 if len(value.shape) == 1:
#                     # 1D arrays can be directly added
#                     data_dict[attr_name] = value
#                     if base_length is None:
#                         base_length = len(value)
#                 else:
#                     # For multi-dimensional arrays, store them as objects
#                     # Each row will contain a slice of the array
#                     data_dict[attr_name] = [value[i] for i in range(value.shape[0])]
#                     if base_length is None:
#                         base_length = value.shape[0]
#
#         # Create DataFrame
#         df = pd.concat([pd.DataFrame(data_dict),self.brain_location], axis=1, sort=False)
#
#         return df
#
# @dataclass
# class Wheel(BaseLoader):
#     """Represents wheel data."""
#     times: Optional[np.ndarray] = field(default=None)
#     positions: Optional[np.ndarray] = field(default=None)
#
#     @classmethod
#     def get_file_mapping(cls) -> Dict[str, str]:
#         return {
#             'wheel.timestamps.npy': 'times',
#             'wheel.position.npy': 'positions',
#         }
#
#     @classmethod
#     def from_tar(cls, tar_path: str | Path) -> 'BaseLoader':
#         """Load data from a tar archive."""
#         instance = cls()
#         file_attr_map = cls.get_file_mapping()
#
#         with tarfile.open(tar_path, 'r') as tar:
#             for file_name, attr_name in file_attr_map.items():
#                 if file_name not in tar.getnames():
#                     print(f"Warning: {file_name} not found in tar archive")
#                     continue
#                 if attr_name == 'times':
#                     setattr(instance, attr_name, np.squeeze(npy_loader(tar, file_name)))
#                 else:
#                     setattr(instance, attr_name, np.squeeze(npy_loader(tar, file_name)))
#         return instance
#
#     def to_dataframe(self) -> pd.DataFrame:
#         """
#         Convert the Wheel data to a pandas DataFrame.
#         For multi-dimensional arrays, only the first dimension is used as the index,
#         and the remaining dimensions are stored as array objects in the cells.
#         """
#         data_dict = {}
#         base_length = None
#
#         # Process each attribute
#         for attr_name, value in self.__dict__.items():
#             if attr_name == 'times':
#                 continue
#             elif value is not None:
#                 if len(value.shape) == 1:
#                     # 1D arrays can be directly added
#                     data_dict[attr_name] = value
#                     if base_length is None:
#                         base_length = len(value)
#                 else:
#                     # For multi-dimensional arrays, store them as objects
#                     # Each row will contain a slice of the array
#                     data_dict[attr_name] = [value[i] for i in range(value.shape[0])]
#                     if base_length is None:
#                         base_length = value.shape[0]
#
#         # Create DataFrame
#         df = pd.DataFrame(data_dict)
#         df['times'] = np.linspace(self.times[0,1],self.times[1,1],int(self.times[1,0]+1))
#         # Constants
#         TICKS_PER_REV = 1440
#         DEGREES_PER_TICK = 360 / TICKS_PER_REV
#
#         # Compute angle position (modulo 1440 to stay within one revolution)
#         df['angle_position'] = df['positions'].apply(
#             lambda x: (x % TICKS_PER_REV) * DEGREES_PER_TICK
#         )
#
#         # Compute current revolution count (integer division)
#         df['current_revolution'] = df['positions'].apply(
#             lambda x: x // TICKS_PER_REV
#         )
#
#         # Compute direction change (+1 forward, -1 backward, 0 if no change)
#         df['angle_change'] = np.diff(df['positions'],
#                                                 prepend=df['positions'][0]) * DEGREES_PER_TICK
#         return df
#
# @dataclass
# class Session(BaseLoader):
#     """Represents session data."""
#     mouse_name: str = field(default=None)
#     date_exp: str = field(default=None)
#     cluster_df: Optional[pd.DataFrame] = field(default=None)
#     spikes_df: Optional[pd.DataFrame] = field(default=None)
#     trials_df: Optional[pd.DataFrame] = field(default=None)
#     wheel_df: Optional[pd.DataFrame] = field(default=None)
#     channels_df: Optional[pd.DataFrame] = field(default=None)
#
#     @classmethod
#     def get_file_mapping(cls) -> Dict[str, str]:
#         return {
#             'Clusters':'cluster_df',
#             'Spikes':'spikes_df',
#             'Trials':'trials_df',
#             'Wheels':'wheel_df',
#             'Channels':'channels_df',
#         }
#
#     @classmethod
#     def from_tar(cls, tar_path: str | Path) -> 'Session':
#         """Load data from a tar archive and return a Session instance."""
#         instance = cls()
#         file_attr_map = cls.get_file_mapping()
#
#         setattr(instance, 'mouse_name', Path(tar_path).stem.split('_')[0])
#         setattr(instance, 'date_exp', Path(tar_path).stem.split('_')[1])
#
#         for file_name, attr_name in file_attr_map.items():
#             if file_name == 'Clusters':
#                 session_data = Clusters.from_tar(tar_path)
#                 setattr(instance, attr_name, session_data.to_dataframe())
#                 instance.cluster_df = session_data.to_dataframe().assign(
#                     mouse_name=instance.mouse_name,
#                     date_exp=instance.date_exp
#                 )
#             elif file_name == 'Spikes':
#                 session_data = Spikes.from_tar(tar_path)
#                 setattr(instance, attr_name, session_data.to_dataframe())
#                 instance.spikes_df = session_data.to_dataframe().assign(
#                     mouse_name=instance.mouse_name,
#                     date_exp=instance.date_exp
#                 )
#             elif file_name == 'Trials':
#                 session_data = Trials.from_tar(tar_path)
#                 setattr(instance, attr_name, session_data.to_dataframe())
#                 instance.trials_df = session_data.to_dataframe().assign(
#                     mouse_name=instance.mouse_name,
#                     date_exp=instance.date_exp
#                 )
#             elif file_name == 'Wheels':
#                 session_data = Wheel.from_tar(tar_path)
#                 setattr(instance, attr_name, session_data.to_dataframe())
#                 instance.wheel_df = session_data.to_dataframe().assign(
#                     mouse_name=instance.mouse_name,
#                     date_exp=instance.date_exp
#                 )
#             elif file_name == 'Channels':
#                 session_data = Channels.from_tar(tar_path)
#                 setattr(instance, attr_name, session_data.to_dataframe())
#                 instance.channels_df = session_data.to_dataframe().assign(
#                     mouse_name=instance.mouse_name,
#                     date_exp=instance.date_exp
#                 )
#
#         closed_sites = instance.channels_df.iloc[np.where(
#             np.diff(instance.channels_df.site.values) != 1)[0]].site.values + 1
#         instance.cluster_df.loc[instance.cluster_df.site.isin(closed_sites), 'site'] -= 1
#         instance.spikes_df = instance.spikes_df.merge(instance.cluster_df,
#                                                     left_on='clusters',
#                                                     right_index=True,
#                                                     how='left').query('phy_annotation != 1')
#
#         spikes_sorted = instance.spikes_df.sort_values('times')
#         trials_sorted = instance.trials_df.sort_values('start')
#         wheel_sorted = instance.wheel_df.sort_values('times')
#
#         # Assign the closest trial start to each spike and wheel position
#         spikes_with_intervals = pd.merge_asof(spikes_sorted, trials_sorted, left_on='times', right_on='start',
#                                               direction='backward').dropna()
#
#         wheel_with_intervals = pd.merge_asof(wheel_sorted, trials_sorted, left_on='times', right_on='start',
#                                              direction='backward').dropna()
#
#         # Keep only spikes within their assigned interval
#         spikes_with_intervals = spikes_with_intervals[spikes_with_intervals['times'] <= spikes_with_intervals['end']]
#         wheel_with_intervals = wheel_with_intervals[wheel_with_intervals['times'] <= wheel_with_intervals['end']]
#
#         instance.spikes_df = spikes_with_intervals
#         instance.wheel_df = wheel_with_intervals
#
#         # instance.cluster_df = instance.cluster_df.merge(instance.channels_df[['probe', 'site', 'ccf_ap', 'ccf_dv', 'ccf_lr', 'allen_ontology']],
#         #                               on=['probe', 'site'], how='left')
#
#         return instance

In [6]:
alldata_tar_path = os.path.join(os.getcwd(),data_directory,test_LFP + r".tar")
print(alldata_tar_path)
session_data = Session.from_tar(alldata_tar_path)

C:\Python Work Directory\NMA_Impact_Scholars_Steinmetz\data\spikeAndBehavioralData\Cori_2016-12-14.tar
<class 'tarfile.TarFile'>
['clusters.depths.npy', 'clusters.originalIDs.npy', 'clusters.peakChannel.npy', 'clusters.probes.npy', 'clusters.templateWaveformChans.npy', 'clusters.templateWaveforms.npy', 'clusters.waveformDuration.npy', 'clusters._phy_annotation.npy']
['spikes.amps.npy', 'spikes.clusters.npy', 'spikes.depths.npy', 'spikes.times.npy']
['trials.feedbackType.npy', 'trials.feedback_times.npy', 'trials.goCue_times.npy', 'trials.included.npy', 'trials.intervals.npy', 'trials.repNum.npy', 'trials.response_choice.npy', 'trials.response_times.npy', 'trials.visualStim_contrastLeft.npy', 'trials.visualStim_contrastRight.npy', 'trials.visualStim_times.npy']
['wheel.position.npy', 'wheel.timestamps.npy', 'wheelMoves.intervals.npy', 'wheelMoves.type.npy']
depths (1085,)
original_ids (1085,)
site (1085,)
probe (1085,)
template_waveform_chans (1085, 50)
template_waveforms (1085, 82, 50)

In [7]:
print(session_data.mouse_name, session_data.date_exp)

Cori 2016-12-14


In [8]:
wheel_df = wheel_data.to_dataframe()
wheel_df.head()

,positions,times,angle_position,current_revolution,angle_change
0,0.0,33.652504,0.0,0.0,0.0
1,0.0,33.652904,0.0,0.0,0.0
2,0.0,33.653304,0.0,0.0,0.0
3,0.0,33.653704,0.0,0.0,0.0
4,0.0,33.654104,0.0,0.0,0.0


In [9]:
wheel_df.head(500)

,positions,times,angle_position,current_revolution,angle_change
0,0.0,33.652504,0.0,0.0,0.0
1,0.0,33.652904,0.0,0.0,0.0
2,0.0,33.653304,0.0,0.0,0.0
3,0.0,33.653704,0.0,0.0,0.0
4,0.0,33.654104,0.0,0.0,0.0
...,...,...,...,...,...
495,0.0,33.850507,0.0,0.0,0.0
496,0.0,33.850907,0.0,0.0,0.0
497,0.0,33.851307,0.0,0.0,0.0
498,0.0,33.851707,0.0,0.0,0.0


In [10]:
cluster_df = session_data.cluster_df
print(cluster_df.shape)
cluster_df = cluster_df.query('phy_annotation != 0')
cluster_df.phy_annotation.value_counts()

(1085, 11)


phy_annotation
3.0    729
1.0    351
2.0      5
Name: count, dtype: int64

In [11]:
# cluster_df[['mouse_name', 'date_exp']] = session_label
cluster_df.head()

,depths,original_ids,site,probe,template_waveform_chans,template_waveforms,waveform_duration,phy_annotation,peak_to_trough_duration,mouse_name,date_exp
0,2297.232386,0,224.0,0.0,"[223.0, 225.0, 222.0, 220.0, 227.0, 219.0, 221...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",19.0,3.0,0.600000,Cori,2016-12-14
1,3240.539945,1,316.0,0.0,"[315.0, 317.0, 313.0, 316.0, 312.0, 314.0, 311...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",19.0,3.0,0.666667,Cori,2016-12-14
2,1744.355208,3,172.0,0.0,"[171.0, 169.0, 168.0, 167.0, 170.0, 173.0, 166...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",10.0,3.0,0.300000,Cori,2016-12-14
3,265.098545,4,28.0,0.0,"[27.0, 24.0, 25.0, 26.0, 23.0, 22.0, 28.0, 29....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",17.0,3.0,0.533333,Cori,2016-12-14
4,328.853292,7,34.0,0.0,"[33.0, 31.0, 29.0, 32.0, 35.0, 28.0, 30.0, 27....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",16.0,1.0,0.533333,Cori,2016-12-14


In [12]:
(cluster_df.phy_annotation.values != 1)[:100]

array([ True,  True,  True,  True, False, False,  True,  True, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False, False,  True, False,  True,  True, False,  True,  True,
        True,  True, False,  True, False, False,  True,  True,  True,
        True,  True, False,  True, False,  True, False,  True, False,
        True, False,  True,  True,  True, False,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True, False,
       False, False, False, False,  True,  True,  True, False,  True,
        True,  True, False,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True, False,  True,  True,  True,  True,
        True])

In [13]:
trial_df = session_data.trials_df
trial_df.head()

,feedback_type,feedback_times,gocue_times,included,intervals,repNum,response_choice,response_times,contrast_left,contrast_right,stimulus_times,start,end,mouse_name,date_exp
0,1.0,66.456227,66.296625,True,"[62.90028371085413, 67.42348445160172]",1.0,1.0,66.419612,1.0,0.0,65.269408,62.900284,67.423484,Cori,2016-12-14
1,1.0,72.640326,72.077117,True,"[68.42083771830875, 73.60447564732587]",1.0,-1.0,72.602206,0.0,0.5,71.202703,68.420838,73.604476,Cori,2016-12-14
2,1.0,77.038396,76.877593,True,"[74.60290237095717, 78.00675700882503]",1.0,1.0,77.001671,1.0,0.5,76.052380,74.602902,78.006757,Cori,2016-12-14
3,1.0,83.531699,81.996875,True,"[79.00365324125164, 84.50677806190824]",1.0,0.0,83.502065,0.0,0.0,81.235263,79.003653,84.506778,Cori,2016-12-14
4,-1.0,87.628565,87.462962,True,"[85.50179514075045, 88.62133641478076]",1.0,1.0,87.617727,0.5,1.0,86.800952,85.501795,88.621336,Cori,2016-12-14


In [14]:
spike_df = session_data.spikes_df
spike_df.head()

,amps,times,clusters,depths_x,mouse_name_x,date_exp_x,depths_y,original_ids,site,probe,...,repNum,response_choice,response_times,contrast_left,contrast_right,stimulus_times,start,end,mouse_name,date_exp
157851,73.718240,62.900322,1025,395.479126,Cori,2016-12-14,405.177059,1007,412.0,1.0,...,1.0,1.0,66.419612,1.0,0.0,65.269408,62.900284,67.423484,Cori,2016-12-14
157852,166.596692,62.901588,1033,182.729370,Cori,2016-12-14,177.302407,1015,392.0,1.0,...,1.0,1.0,66.419612,1.0,0.0,65.269408,62.900284,67.423484,Cori,2016-12-14
157853,343.698088,62.901688,1021,3213.779541,Cori,2016-12-14,3223.968454,1000,687.0,1.0,...,1.0,1.0,66.419612,1.0,0.0,65.269408,62.900284,67.423484,Cori,2016-12-14
157854,155.385044,62.902088,1035,180.991135,Cori,2016-12-14,183.813050,1017,392.0,1.0,...,1.0,1.0,66.419612,1.0,0.0,65.269408,62.900284,67.423484,Cori,2016-12-14
157855,109.415228,62.902288,1054,3072.125244,Cori,2016-12-14,3090.440767,1038,672.0,1.0,...,1.0,1.0,66.419612,1.0,0.0,65.269408,62.900284,67.423484,Cori,2016-12-14


In [15]:
channel_df = session_data.channels_df
channel_df.head(500)

,site_positions,site,probe,raw_row,ccf_ap,ccf_dv,ccf_lr,allen_ontology,mouse_name,date_exp
0,"[43.0, 20.0]",0,0.0,0,4432.3,4699.7,5002.3,LS,Cori,2016-12-14
1,"[11.0, 20.0]",1,0.0,1,4428.4,4699.7,4970.6,LS,Cori,2016-12-14
2,"[59.0, 40.0]",2,0.0,2,4431.3,4680.0,5018.6,LS,Cori,2016-12-14
3,"[27.0, 40.0]",3,0.0,3,4427.3,4680.0,4986.8,LS,Cori,2016-12-14
4,"[43.0, 60.0]",4,0.0,4,4426.3,4660.2,5003.1,LS,Cori,2016-12-14
...,...,...,...,...,...,...,...,...,...,...
495,"[43.0, 1260.0]",124,1.0,124,9044.8,3516.6,2920.8,root,Cori,2016-12-14
496,"[11.0, 1260.0]",125,1.0,125,9064.3,3516.6,2946.2,root,Cori,2016-12-14
497,"[59.0, 1280.0]",126,1.0,126,9039.4,3497.4,2904.8,root,Cori,2016-12-14
498,"[27.0, 1280.0]",127,1.0,127,9058.9,3497.4,2930.2,root,Cori,2016-12-14


In [16]:
channel_df.iloc[np.where(np.diff(channel_df.site.values) != 1)[0]]

,site_positions,site,probe,raw_row,ccf_ap,ccf_dv,ccf_lr,allen_ontology,mouse_name,date_exp
35,"[27.0, 360.0]",35,0.0,35,4379.9,4363.6,4992.7,LS,Cori,2016-12-14
73,"[59.0, 760.0]",74,0.0,74,4324.4,3968.1,5031.8,LS,Cori,2016-12-14
109,"[27.0, 1120.0]",111,0.0,111,4267.1,3612.1,5006.6,LS,Cori,2016-12-14
147,"[59.0, 1520.0]",150,0.0,150,4211.6,3216.6,5045.7,root,Cori,2016-12-14
183,"[27.0, 1880.0]",187,0.0,187,4154.3,2860.7,5020.5,ACA,Cori,2016-12-14
221,"[59.0, 2280.0]",226,0.0,226,4098.8,2465.2,5059.6,ACA,Cori,2016-12-14
257,"[27.0, 2640.0]",263,0.0,263,4041.5,2109.2,5034.5,ACA,Cori,2016-12-14
295,"[59.0, 3040.0]",302,0.0,302,3986.0,1713.7,5073.6,MOs,Cori,2016-12-14
331,"[27.0, 3400.0]",339,0.0,339,3928.7,1357.8,5048.4,MOs,Cori,2016-12-14
369,"[59.0, 3800.0]",378,0.0,378,3873.3,962.2,5087.5,root,Cori,2016-12-14


In [17]:
channel_df.iloc[np.where(np.diff(channel_df.site.values) != 1)[0]+1]

,site_positions,site,probe,raw_row,ccf_ap,ccf_dv,ccf_lr,allen_ontology,mouse_name,date_exp
36,"[11.0, 380.0]",37,0.0,37,4374.9,4343.8,4977.2,LS,Cori,2016-12-14
74,"[43.0, 780.0]",76,0.0,76,4319.5,3948.3,5016.3,LS,Cori,2016-12-14
110,"[11.0, 1140.0]",113,0.0,113,4262.1,3592.3,4991.1,LS,Cori,2016-12-14
148,"[43.0, 1540.0]",152,0.0,152,4206.7,3196.8,5030.2,root,Cori,2016-12-14
184,"[11.0, 1900.0]",189,0.0,189,4149.3,2840.9,5005.0,ACA,Cori,2016-12-14
222,"[43.0, 2300.0]",228,0.0,228,4093.9,2445.4,5044.1,ACA,Cori,2016-12-14
258,"[11.0, 2660.0]",265,0.0,265,4036.6,2089.4,5019.0,ACA,Cori,2016-12-14
296,"[43.0, 3060.0]",304,0.0,304,3981.1,1693.9,5058.0,MOs,Cori,2016-12-14
332,"[11.0, 3420.0]",341,0.0,341,3923.8,1338.0,5032.9,MOs,Cori,2016-12-14
370,"[43.0, 3820.0]",380,0.0,380,3868.3,942.5,5072.0,root,Cori,2016-12-14


In [18]:
# closed_sites =  channel_df.iloc[np.where(np.diff(channel_df.site.values) != 1)[0]].site.values+1

In [19]:
# cluster_df.loc[cluster_df['site'].isin(closed_sites), 'site'] -= 1

In [20]:
# Using merge
# cluster_df = cluster_df.merge(channel_df[['probe', 'site','ccf_ap','ccf_dv','ccf_lr','allen_ontology']], on=['probe','site'], how='left')

In [21]:
cluster_df.head()

,depths,original_ids,site,probe,template_waveform_chans,template_waveforms,waveform_duration,phy_annotation,peak_to_trough_duration,mouse_name,date_exp
0,2297.232386,0,224.0,0.0,"[223.0, 225.0, 222.0, 220.0, 227.0, 219.0, 221...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",19.0,3.0,0.600000,Cori,2016-12-14
1,3240.539945,1,316.0,0.0,"[315.0, 317.0, 313.0, 316.0, 312.0, 314.0, 311...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",19.0,3.0,0.666667,Cori,2016-12-14
2,1744.355208,3,172.0,0.0,"[171.0, 169.0, 168.0, 167.0, 170.0, 173.0, 166...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",10.0,3.0,0.300000,Cori,2016-12-14
3,265.098545,4,28.0,0.0,"[27.0, 24.0, 25.0, 26.0, 23.0, 22.0, 28.0, 29....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",17.0,3.0,0.533333,Cori,2016-12-14
4,328.853292,7,34.0,0.0,"[33.0, 31.0, 29.0, 32.0, 35.0, 28.0, 30.0, 27....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",16.0,1.0,0.533333,Cori,2016-12-14


In [31]:
# Step 2: Extract and Visualize Data from alldat.
# Load Data
fnames = []
for j in range(1,4,1):
  fnames.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fnames[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fnames[j], "wb") as fid:
          fid.write(r.content)

# Function to load data from npz files
def load_data(filenames):
    alldat = np.array([])
    for filename in filenames:
        data = np.load(filename, allow_pickle=True)['dat']
        alldat = np.hstack((alldat, data))
    return alldat

# Download data if not already present
alldat = load_data(fnames)

In [32]:
walker = os.walk(os.path.join(os.getcwd(),data_directory))
for _, _, files in walker:
    for file in files:
        mouse_name = Path(file).stem.split("_")[0]
        date_exp = Path(file).stem.split("_")[1]
        print(mouse_name, date_exp)

Cori 2016-12-14
Cori 2016-12-17
Cori 2016-12-18
Forssmann 2017-11-01
Forssmann 2017-11-02
Forssmann 2017-11-04
Forssmann 2017-11-05
Hench 2017-06-15
Hench 2017-06-16
Hench 2017-06-17
Hench 2017-06-18
Lederberg 2017-12-05
Lederberg 2017-12-06
Lederberg 2017-12-07
Lederberg 2017-12-08
Lederberg 2017-12-09
Lederberg 2017-12-10
Lederberg 2017-12-11
Moniz 2017-05-15
Moniz 2017-05-16
Moniz 2017-05-18
Muller 2017-01-07
Muller 2017-01-08
Muller 2017-01-09
Radnitz 2017-01-08
Radnitz 2017-01-09
Radnitz 2017-01-10
Radnitz 2017-01-11
Radnitz 2017-01-12
Richards 2017-10-29
Richards 2017-10-30
Richards 2017-10-31
Richards 2017-11-01
Richards 2017-11-02
Tatum 2017-12-06
Tatum 2017-12-07
Tatum 2017-12-08
Tatum 2017-12-09
Theiler 2017-10-11
